### Generación de datasets de prueba para CI (base y “futuro”)

**Objetivo.** Crear dos CSV que usará el CI (pytest) para evaluar al modelo:
- `app/data/titanic_test_base.csv` → cohorte “estable”.
- `app/data/titanic_test_future.csv` → cohorte con **cambio de distribución** (drift).

**Qué hace este bloque:**
1. **Carga el dataset** *titanic* desde OpenML y define las columnas de entrada (`features`) y la etiqueta (`survived` como 0/1).
2. **Repite el split** de entrenamiento (mismo `random_state` y `stratify`) para obtener un `X_test` comparable con el del notebook 00.
3. **Guarda** `titanic_test_base.csv` uniendo `X_test` + `survived`.
4. **Construye el set “futuro”** forzando un sesgo (ejemplo: solo **3ª clase** `pclass==3`) para simular drift.
   - Si queda muy grande, hace un **sample** hasta 300 filas (con `random_state` fijo).
5. **Guarda** `titanic_test_future.csv` con las mismas columnas + `survived`.
6. Muestra en pantalla las **rutas y shapes** finales de ambos CSV.

**Cómo se usa en CI.**
Los tests en `app/tests/` leen estas rutas y calculan F1:
- Base → debe superar el umbral.
- Futuro → puede bajar (sirve como señal de drift).

> Si se quiere simular **otro tipo de drift**, cambia el filtro del set futuro (por ejemplo, solo `sex=="male"`, alterar rangos de `age`, etc.) manteniendo el esquema de columnas.


In [2]:
import os
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# 1) Cargar dataset crudo (mismas columnas que usa tu Pipeline)
X_titanic, y_titanic = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
y_titanic = y_titanic.astype(int)  # aseguramos 0/1

# 2) Hacer el split igual que en el 00 (mismo random_state y stratify)
X_train, X_test, y_train, y_test = train_test_split(
    X_titanic[features], y_titanic,
    test_size=0.2, random_state=42, stratify=y_titanic
)

# 3) Crear carpeta de salida
os.makedirs("app/data", exist_ok=True)

# 4) CSV de test "base"
test_base = X_test.copy()
test_base["survived"] = y_test.values
test_base.to_csv("app/data/titanic_test_base.csv", index=False)

# 5) CSV de test "futuro" (ejemplo: solo 3ª clase para cambiar la distribución)
mask_pclass3 = (X_titanic["pclass"] == 3)
test_future = X_titanic.loc[mask_pclass3, features].copy()
test_future["survived"] = y_titanic.loc[mask_pclass3].values

# Si queda muy grande, samplea
test_future = test_future.sample(n=min(300, len(test_future)), random_state=123)
test_future.to_csv("app/data/titanic_test_future.csv", index=False)

print("✅ Guardados:")
print(" - app/data/titanic_test_base.csv  ", test_base.shape)
print(" - app/data/titanic_test_future.csv", test_future.shape)


✅ Guardados:
 - app/data/titanic_test_base.csv   (262, 8)
 - app/data/titanic_test_future.csv (300, 8)
